In [31]:
import pandas as pd
from prettytable import PrettyTable
from nltk.tokenize import word_tokenize
import nltk.stem.porter as porter
from def_overlap_funcs import filter_words, similarity_score

## Similarità definizioni
Si sono create, per ogni studente, quattro definizioni. Il task successivo richiedeva di valutare la similarità tra le definizioni, per ognuno dei concetti scelti.
Il primo output visibile nel notebook rappresenta la tabella con un concetto per colonna e nelle righe le definizioni pulite da stop words e punteggiatura e stemmatizzate (ovvero ridotte alla loro *radice*).

In [32]:
stemmer = porter.PorterStemmer()

In [33]:
data = pd.read_csv('TLN-definitions-23.csv', sep=';')
data = data.applymap(lambda s: filter_words(set(word_tokenize(s))))
data = data.applymap(lambda tokens: {stemmer.stem(t) for t in tokens})

data

,door,ladybug,pain,blurriness
0,"{passag, room, construct, two, use, temporaril...","{insect, black, leg, six, typic, red, spot, sm...","{distress, mental, physic, feel}","{focu, sight}"
1,"{close, open}","{insect, luck, black, symbol, red, dot, good, ...","{feel, bad, hurt, emot, physic}","{absenc, definit, shapeless, border}"
2,"{peopl, room, wall, object, get, let, two, hol...","{insect, black, red, color, orang, dot, fli}","{experi, suffer, consciou, fell, be}","{felt, sensat, someth, clearli, see, border, ca}"
3,"{one, usabl, anoth, area, access}","{back, small, insect, red}","{live, concept, describ, suffer}","{lack, sharp}"
4,"{structur, allow, delimit, area, access}","{round, fli, small, insect}","{discomfort, physic, feel}","{lack, characterist, clariti, precis}"
5,"{passag, object, move, block, pass, use}","{beatl, insect, famili, red, small, color, ten...","{sensat, burn, unpleas}","{blur, sensat, eye, see, everyth}"
6,"{separ, main, histor, also, wood, build, iron,...","{natur, insect, luck, insectivor, black, consi...","{felt, creatur, natur, sensat, damag, bodi, ps...","{perceiv, whether, goal, conceptu, kind, inabl..."
7,"{room, object, separ, close, use, wall, go, open}","{insect, luck, black, red, bring, small, dot, ...","{pain, feel, caus, injuri, sad, emot, physic}","{lack, eye, vision, imag, bad, unfocus, memori}"
8,"{order, someth, anoth, access, place, open}","{insect, black, red, small, dot}","{sorrow, express, sentiment}","{clear, total, thant, see, make, condit, bright}"
9,"{room, access}","{coat, insect, yellow, red, small, dot}","{sensat, exert, unpleas, result, emot, physic}","{exert, state, use, result, confus, alcohol, p..."


### Calcolo punteggi similarità tra definizioni
In questa cella vengono calcolate le similarità tra le definizioni di una colonna. Il processo viene eseguito per ogni colonna e i per ognuna i valori sono salvati in una lista

In [34]:
scores = []
words = list(data.columns.values)
for word in words:
    word_scores = [similarity_score(len(def1.intersection(def2)), len(def1), len(def2))
                   for def1 in data[word] for def2 in data[word] if def1 != def2]
    scores.append(word_scores)

### Valore medio
Per sapere il valore medio di similarità tra le definizioni di una colonna si fa il rapporto tra la somma dei valori e il numero di valori. I risultati sono mostrati nell'output sottostante .
Il punteggio di similarità più alta lo si trova per la parola **'ladybug'** con un valore di 0.5967. Per i concetti generici, *'door'* e *'pain'*, i punteggi sono molto simili e sono poco sopra allo 0.20. Il punteggio più basso lo si trova per il concetto *'blurriness'* con uno score di 0.081.

In [41]:
results = [round(sum(word_scores) / len(word_scores), 4) for word_scores in scores]

print('            Astratto    Concreto')
print('Generico    ', results[2], '   ',results[0])
print('Specifico   ', results[3], '    ', results[1])

            Astratto    Concreto
Generico     0.2524     0.2345
Specifico    0.081      0.5967
